# 损失函数类

损失函数是前向传播链路的终点，也是梯度计算链路的起点。

我们要把损失函数封装进**损失函数**（Loss）类。它将和张量类、层类一起，实现前向传播和梯度计算的逻辑闭环。这个闭环，称为**计算图**（Computational Graph）。

## 计算图

我们已经知道，神经网络模型并不是一次性地从输入数据计算出输出数据。而是输入数据经过一层一层的神经元，通过多次（简单）计算最终得出输出数据。这个数据流通的线路，被称为前向传播。

在前向传播的过程中，我们需要记录下来输入数据流通的整个**拓扑结构**。因为我们需要通过这个拓扑结构反向地逐级计算梯度。这个数据前向传播的拓扑结构，称为**计算图**。

计算图从特征值开始，沿着层结构，最终汇合到损失函数。计算图中的每个节点都由张量构成，可能是参数，可能是中间值；最后一个节点通常是损失值。

因为它是在前向传播的过程中动态构建起来的，所以称为**动态计算图**（Dynamic Computational Graph）；相对应的，在前向传播进行前，就先构建好的计算图，称为**静态计算图**（Static Computational Graph）。PyTorch采用动态计算图技术，而TensorFlow则采用静态计算图技术。我们将采用动态计算图技术。

In [11]:
from abc import abstractmethod, ABC
import numpy as np

## 基础架构

### 张量

In [12]:
class Tensor:

    def __init__(self, data):
        self.data = np.array(data)

    @property
    def size(self):
        return len(self.data)

    def __repr__(self):
        return f'Tensor({self.data})'

### 基础层

In [13]:
class Layer(ABC):

    def __call__(self, x: Tensor):
        return self.forward(x)

    @abstractmethod
    def forward(self, x: Tensor):
        pass

    def __repr__(self):
        return ''

### 基础损失函数

和基础层一样，**基础损失函数**（Base Loss）也是一个抽象类，定义了一个**计算损失**（loss）的虚拟接口。这个接口需要返回一个损失值的张量，作为梯度计算的起点。

In [14]:
class Loss(ABC):

    def __call__(self, p: Tensor, y: Tensor):
        return self.loss(p, y)

    @abstractmethod
    def loss(self, p: Tensor, y: Tensor):
        pass

## 数据

### 特征、标签

我们需要提供**标签值**，这样损失函数才能计算损失值。

In [15]:
feature = Tensor([28.1, 58.0])
label = Tensor([165])

## 模型

### 线性层

In [16]:
class Linear(Layer):

    def __init__(self, in_size, out_size):
        self.weight = Tensor(np.ones((out_size, in_size)) / in_size)
        self.bias = Tensor(np.zeros(out_size))

    def forward(self, x: Tensor):
        return Tensor(x.data @ self.weight.data.T + self.bias.data)

    def __repr__(self):
        return f'Linear[weight{self.weight.data.shape}; bias{self.bias.data.shape}]'

### 损失函数（均方误差）

我们定义的第一个损失函数类是**均方误差损失函数**（MSELoss）。它封装了计算均方误差的功能，这是数值回归模型最常用的损失函数。

In [17]:
class MSELoss(Loss):

    def loss(self, p: Tensor, y: Tensor):
        return Tensor(np.mean(np.square(y.data - p.data)))

## 验证

### 建模

In [18]:
# 层
layer = Linear(feature.size, 1)
print(layer)
# 损失函数
loss_fn = MSELoss()

Linear[weight(1, 2); bias(1,)]


### 推理

In [19]:
prediction = layer(feature)
print(f'prediction: {prediction}')

prediction: Tensor([43.05])


### 评估

In [20]:
loss = loss_fn(prediction, label)
print(f'loss: {loss}')

loss: Tensor(14871.802500000002)


损失函数类实现了前向传播链路和梯度计算链路的闭环。下一步，我们将在此基础上，实现自动微分和动态计算图，完成最复杂的梯度计算的自动化。

## 课后练习

试试创建一个新的损失函数类：**平均绝对误差损失函数**（MAELoss）。它是数值回归模型中，另一种常用的损失函数。它的公式是：

   $$
   \text{mean absolute error} = \frac{1}{n} \sum_{i=1}^{n} |y_i - p_i|
   $$